### BioGrid

In [2]:
# Download Link: https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-4.4.243/BIOGRID-ALL-4.4.243.mitab.zip
# Download Date: 2025-03-21
# Download Version: 2025-03-02

import pandas as pd

df_biogrid = pd.read_csv('BIOGRID-ALL-4.4.243.mitab.txt', sep='\t')
df_biogrid_filter = df_biogrid[(df_biogrid['Taxid Interactor A'] == 'taxid:9606') & (df_biogrid['Taxid Interactor B'] == 'taxid:9606')]
df_biogrid_filter_relation = df_biogrid_filter[['Alt IDs Interactor A', 'Alt IDs Interactor B']]
df_biogrid_filter_relation.drop_duplicates(inplace=True)
df_biogrid_filter_relation

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\1415521415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_biogrid_filter_relation.drop_duplicates(inplace=True)


,Alt IDs Interactor A,Alt IDs Interactor B
0,biogrid:112315|entrez gene/locuslink:MAP2K4|un...,biogrid:108607|entrez gene/locuslink:FLNC|unip...
1,biogrid:124185|entrez gene/locuslink:MYPN|unip...,biogrid:106603|entrez gene/locuslink:ACTN2|uni...
2,biogrid:106605|entrez gene/locuslink:ACVR1|uni...,biogrid:108625|entrez gene/locuslink:FNTA|unip...
3,biogrid:108894|entrez gene/locuslink:GATA2|uni...,biogrid:111384|entrez gene/locuslink:PML|unipr...
4,biogrid:112038|entrez gene/locuslink:RPA2|entr...,biogrid:112651|entrez gene/locuslink:STAT3|uni...
...,...,...
2752007,biogrid:109512|entrez gene/locuslink:PRMT1|uni...,biogrid:106835|entrez gene/locuslink:APOB|unip...
2752009,biogrid:109512|entrez gene/locuslink:PRMT1|uni...,biogrid:115581|entrez gene/locuslink:TRAIP|uni...
2752011,biogrid:116544|entrez gene/locuslink:MLXIP|uni...,biogrid:109512|entrez gene/locuslink:PRMT1|uni...
2752014,biogrid:110092|entrez gene/locuslink:KTN1|unip...,biogrid:109512|entrez gene/locuslink:PRMT1|uni...


In [3]:
def extract_uniprot_ids(data):
    return data.apply(lambda x: '|'.join([entry for entry in x.split('|') if 'uniprot' in entry]))

df_biogrid_filter_relation['Alt IDs Interactor A'] = extract_uniprot_ids(df_biogrid_filter_relation['Alt IDs Interactor A'])
df_biogrid_filter_relation['Alt IDs Interactor B'] = extract_uniprot_ids(df_biogrid_filter_relation['Alt IDs Interactor B'])

df_biogrid_filter_relation['Alt IDs Interactor A'] = df_biogrid_filter_relation['Alt IDs Interactor A'].str.replace('uniprot/swiss-prot:', '')
df_biogrid_filter_relation['Alt IDs Interactor B'] = df_biogrid_filter_relation['Alt IDs Interactor B'].str.replace('uniprot/swiss-prot:', '')
df_biogrid_filter_relation

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2243967021.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_biogrid_filter_relation['Alt IDs Interactor A'] = extract_uniprot_ids(df_biogrid_filter_relation['Alt IDs Interactor A'])
C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2243967021.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_biogrid_filter_relation['Alt IDs Interactor B'] = extract_uniprot_ids(df_biogrid_filter_relation['Alt IDs Interactor B'])
C:\Users\shunn\AppData\Local\Temp\ipyk

,Alt IDs Interactor A,Alt IDs Interactor B
0,P45985,Q14315
1,Q86TC9,P35609
2,Q04771,P49354
3,P23769,P29590
4,P15927,P40763
...,...,...
2752007,Q99873,P04114
2752009,Q99873,Q9BWF2
2752011,Q9HAP2,Q99873
2752014,Q86UP2,Q99873


In [4]:
df_a_expanded = df_biogrid_filter_relation['Alt IDs Interactor A'].str.split('|', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='Alt IDs Interactor A')
df_b_expanded = df_biogrid_filter_relation['Alt IDs Interactor B'].str.split('|', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='Alt IDs Interactor B')

df_biogrid_filter_relation = df_a_expanded.join(df_b_expanded['Alt IDs Interactor B'])
df_biogrid_filter_relation.dropna(subset=['Alt IDs Interactor A', 'Alt IDs Interactor B'], inplace=True)
df_biogrid_filter_relation = df_biogrid_filter_relation[(df_biogrid_filter_relation['Alt IDs Interactor A'].notna()) & 
                                   (df_biogrid_filter_relation['Alt IDs Interactor A'] != '') & 
                                   (df_biogrid_filter_relation['Alt IDs Interactor B'].notna()) & 
                                   (df_biogrid_filter_relation['Alt IDs Interactor B'] != '')]
df_biogrid_filter_relation.drop(columns=['index'], inplace=True)
df_biogrid_filter_relation.drop_duplicates(inplace=True)
df_biogrid_filter_relation.reset_index(drop=True, inplace=True)
df_biogrid_filter_relation

,Alt IDs Interactor A,Alt IDs Interactor B
0,P45985,Q14315
1,Q86TC9,P35609
2,Q04771,P49354
3,P23769,P29590
4,P15927,P40763
...,...,...
854729,Q99873,Q14152
854730,Q99873,Q7Z406
854731,Q9HAP2,Q9UBB9
854732,Q86UP2,Q9H0D6


In [5]:
df_biogrid_filter_relation_reversed = df_biogrid_filter_relation.copy()
df_biogrid_filter_relation_reversed.rename(columns={'Alt IDs Interactor A': 'Alt IDs Interactor B', 'Alt IDs Interactor B': 'Alt IDs Interactor A'}, inplace=True)
df_biogrid_filter_relation_final = pd.concat([df_biogrid_filter_relation, df_biogrid_filter_relation_reversed], ignore_index=True)
df_biogrid_filter_relation_final.drop_duplicates(inplace=True)
df_biogrid_filter_relation_final.reset_index(drop=True, inplace=True)
df_biogrid_filter_relation_final

,Alt IDs Interactor A,Alt IDs Interactor B
0,P45985,Q14315
1,Q86TC9,P35609
2,Q04771,P49354
3,P23769,P29590
4,P15927,P40763
...,...,...
1690117,Q14152,Q99873
1690118,Q7Z406,Q99873
1690119,Q9UBB9,Q9HAP2
1690120,Q9H0D6,Q86UP2


### STRING

In [6]:
# Download Link: https://stringdb-downloads.org/download/protein.links.full.v12.0/9606.protein.links.full.v12.0.txt.gz
# Download Date: 2025-03-21
# Download Version: 2023-05-16

df_stringdb = pd.read_csv('9606.protein.links.full.v12.0.txt', sep=' ')
df_stringdb_filter = df_stringdb[['protein1', 'protein2']]
df_stringdb_filter['protein1'] = df_stringdb_filter['protein1'].replace('9606.', '', regex=True)
df_stringdb_filter['protein2'] = df_stringdb_filter['protein2'].replace('9606.', '', regex=True)
df_stringdb_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\1076345280.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter['protein1'] = df_stringdb_filter['protein1'].replace('9606.', '', regex=True)
C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\1076345280.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter['protein2'] = df_stringdb_filter['protein2'].replace('9606.', '', regex=True)


,protein1,protein2
0,ENSP00000000233,ENSP00000356607
1,ENSP00000000233,ENSP00000427567
2,ENSP00000000233,ENSP00000253413
3,ENSP00000000233,ENSP00000493357
4,ENSP00000000233,ENSP00000324127
...,...,...
13715399,ENSP00000501317,ENSP00000475489
13715400,ENSP00000501317,ENSP00000370447
13715401,ENSP00000501317,ENSP00000312272
13715402,ENSP00000501317,ENSP00000402092


### KEGG

In [19]:
kegg = pd.read_csv('full_kegg_pathway_list_with_id.csv')
kegg_filter = kegg[(kegg['source_type'] == 'SYMBOL') & (kegg['target_type'] == 'SYMBOL')]
kegg_filter = kegg_filter[['source', 'target']]
kegg_filter.drop_duplicates(inplace=True)
kegg_filter

,source,target
1508,HSD17B8,CBR4
7986,MAN1A2,MGAT1
7987,MAN1B1,MAN1A2
7988,MAN1B1,MAN1A1
7989,MAN1B1,MAN1C1
...,...,...
145760,ACVR2B,NCF2
145762,ACVR2B,RAC1
145764,ACVR2B,RAC2
145766,ACVR2B,RAC3


### BioMedGraphica ID

In [7]:
import pandas as pd
import os
from pathlib import Path

current_working_dir = Path(os.getcwd()).resolve()
grandparent_dir = current_working_dir.parent.parent.parent
target_dir_protein = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Protein' / 'BioMedGraphica_Protein.csv'
biomedgraphica_protein = pd.read_csv(target_dir_protein, dtype=str)

### BioGrid Mapping

In [8]:
uniprot_protein = biomedgraphica_protein[['Uniprot_ID', 'BioMedGraphica_ID']]
uniprot_protein.dropna(subset = ['Uniprot_ID'], inplace=True)
uniprot_protein = uniprot_protein.assign(Uniprot_ID=biomedgraphica_protein['Uniprot_ID'].str.split(';')).explode('Uniprot_ID')

uniprot_protein_to_individualid = uniprot_protein.groupby('Uniprot_ID')['BioMedGraphica_ID'].apply(lambda x: ';'.join(x.dropna().unique())).to_dict()

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2768093953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniprot_protein.dropna(subset = ['Uniprot_ID'], inplace=True)


In [9]:
df_biogrid_filter_relation_final['From_ID'] = df_biogrid_filter_relation_final['Alt IDs Interactor A'].map(uniprot_protein_to_individualid)
df_biogrid_filter_relation_final['To_ID'] = df_biogrid_filter_relation_final['Alt IDs Interactor B'].map(uniprot_protein_to_individualid)
df_biogrid_filter_relation_final

,Alt IDs Interactor A,Alt IDs Interactor B,From_ID,To_ID
0,P45985,Q14315,BMG_PT040844,BMG_PT044958
1,Q86TC9,P35609,BMG_PT068914,BMG_PT040481
2,Q04771,P49354,BMG_PT042738,BMG_PT041044
3,P23769,P29590,BMG_PT039778,BMG_PT040066
4,P15927,P40763,BMG_PT039226,BMG_PT040650
...,...,...,...,...
1690117,Q14152,Q99873,BMG_PT044837,BMG_PT084707
1690118,Q7Z406,Q99873,BMG_PT067828,BMG_PT084707
1690119,Q9UBB9,Q9HAP2,BMG_PT095202,BMG_PT090627
1690120,Q9H0D6,Q86UP2,BMG_PT087968,BMG_PT069229


In [10]:
df_biogrid_filter_relation_final.drop(columns=['Alt IDs Interactor A', 'Alt IDs Interactor B'], inplace=True)

df_biogrid_filter_relation_final.dropna(subset=['From_ID'], inplace=True)
df_biogrid_filter_relation_final.dropna(subset=['To_ID'], inplace=True)
df_biogrid_filter_relation_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1689574 entries, 0 to 1690121
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   From_ID  1689574 non-null  object
 1   To_ID    1689574 non-null  object
dtypes: object(2)
memory usage: 38.7+ MB


In [11]:
df_biogrid_filter_relation_final['From_ID'] = df_biogrid_filter_relation_final['From_ID'].str.split(';')
df_biogrid_filter_relation_final['To_ID'] = df_biogrid_filter_relation_final['To_ID'].str.split(';')

df_biogrid_filter_relation_final = df_biogrid_filter_relation_final.explode('From_ID')
df_biogrid_filter_relation_final = df_biogrid_filter_relation_final.explode('To_ID')

df_biogrid_filter_relation_final.drop_duplicates(inplace=True)
df_biogrid_filter_relation_final.reset_index(drop=True, inplace=True)
df_biogrid_filter_relation_final

,From_ID,To_ID
0,BMG_PT040844,BMG_PT044958
1,BMG_PT068914,BMG_PT040481
2,BMG_PT042738,BMG_PT041044
3,BMG_PT039778,BMG_PT040066
4,BMG_PT039226,BMG_PT040650
...,...,...
1689569,BMG_PT044837,BMG_PT084707
1689570,BMG_PT067828,BMG_PT084707
1689571,BMG_PT095202,BMG_PT090627
1689572,BMG_PT087968,BMG_PT069229


### STRING Mapping

In [12]:
ensembl_protein = biomedgraphica_protein[['Ensembl_Protein_ID', 'BioMedGraphica_ID']]
ensembl_protein.dropna(subset = ['Ensembl_Protein_ID'], inplace=True)
ensembl_protein = ensembl_protein.assign(Ensembl_Protein_ID=biomedgraphica_protein['Ensembl_Protein_ID'].str.split(';')).explode('Ensembl_Protein_ID')

ensembl_protein_to_individualid = ensembl_protein.groupby('Ensembl_Protein_ID')['BioMedGraphica_ID'].apply(lambda x: ';'.join(x.dropna().unique())).to_dict()

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2431755470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ensembl_protein.dropna(subset = ['Ensembl_Protein_ID'], inplace=True)


In [13]:
df_stringdb_filter['From_ID'] = df_stringdb_filter['protein1'].map(ensembl_protein_to_individualid)
df_stringdb_filter['To_ID'] = df_stringdb_filter['protein2'].map(ensembl_protein_to_individualid)
df_stringdb_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\3857375147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter['From_ID'] = df_stringdb_filter['protein1'].map(ensembl_protein_to_individualid)
C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\3857375147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter['To_ID'] = df_stringdb_filter['protein2'].map(ensembl_protein_to_individualid)


,protein1,protein2,From_ID,To_ID
0,ENSP00000000233,ENSP00000356607,BMG_PT042398,BMG_PT069871
1,ENSP00000000233,ENSP00000427567,BMG_PT042398,BMG_PT087566
2,ENSP00000000233,ENSP00000253413,BMG_PT042398,BMG_PT040535
3,ENSP00000000233,ENSP00000493357,BMG_PT042398,BMG_PT084354
4,ENSP00000000233,ENSP00000324127,BMG_PT042398,BMG_PT093927
...,...,...,...,...
13715399,ENSP00000501317,ENSP00000475489,NaN,BMG_PT084447
13715400,ENSP00000501317,ENSP00000370447,NaN,BMG_PT088700
13715401,ENSP00000501317,ENSP00000312272,NaN,BMG_PT083488
13715402,ENSP00000501317,ENSP00000402092,NaN,BMG_PT073360


In [14]:
df_stringdb_filter = df_stringdb_filter[['From_ID', 'To_ID']]
df_stringdb_filter.dropna(subset=['From_ID'], inplace=True)
df_stringdb_filter.dropna(subset=['To_ID'], inplace=True)
df_stringdb_filter.info()

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\1662878631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter.dropna(subset=['From_ID'], inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 13285010 entries, 0 to 13715043
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   From_ID  object
 1   To_ID    object
dtypes: object(2)
memory usage: 304.1+ MB


C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\1662878631.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter.dropna(subset=['To_ID'], inplace=True)


In [15]:
df_stringdb_filter['From_ID'] = df_stringdb_filter['From_ID'].str.split(';')
df_stringdb_filter['To_ID'] = df_stringdb_filter['To_ID'].str.split(';')

df_stringdb_filter = df_stringdb_filter.explode('From_ID')
df_stringdb_filter = df_stringdb_filter.explode('To_ID')

df_stringdb_filter.drop_duplicates(inplace=True)
df_stringdb_filter.reset_index(drop=True, inplace=True)

df_stringdb_filter.info()

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2597415019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter['From_ID'] = df_stringdb_filter['From_ID'].str.split(';')
C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2597415019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stringdb_filter['To_ID'] = df_stringdb_filter['To_ID'].str.split(';')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13193859 entries, 0 to 13193858
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   From_ID  object
 1   To_ID    object
dtypes: object(2)
memory usage: 201.3+ MB


### KEGG Mapping

In [20]:
gene_name_individualid = biomedgraphica_protein[['HGNC_Symbol', 'BioMedGraphica_ID']]
gene_name_individualid.dropna(subset=['HGNC_Symbol'], inplace=True)
gene_name_to_individualid = gene_name_individualid.assign(HGNC_Symbol=gene_name_individualid['HGNC_Symbol'].str.split(';')).explode('HGNC_Symbol')
gene_name_to_individualid = gene_name_to_individualid.groupby('HGNC_Symbol')['BioMedGraphica_ID'].apply(lambda x: ';'.join(x.dropna().unique())).to_dict()

kegg_filter['From_ID'] = kegg_filter['source'].map(gene_name_to_individualid)
kegg_filter['To_ID'] = kegg_filter['target'].map(gene_name_to_individualid)
kegg_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\616423457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_name_individualid.dropna(subset=['HGNC_Symbol'], inplace=True)


,source,target,From_ID,To_ID
1508,HSD17B8,CBR4,BMG_PT078734,BMG_PT072935;BMG_PT119125;BMG_PT120080;BMG_PT1...
7986,MAN1A2,MGAT1,BMG_PT035203;BMG_PT109042;BMG_PT111268;BMG_PT1...,BMG_PT039910;BMG_PT119369;BMG_PT119788;BMG_PT1...
7987,MAN1B1,MAN1A2,BMG_PT097592;BMG_PT129320;BMG_PT129636;BMG_PT1...,BMG_PT035203;BMG_PT109042;BMG_PT111268;BMG_PT1...
7988,MAN1B1,MAN1A1,BMG_PT097592;BMG_PT129320;BMG_PT129636;BMG_PT1...,BMG_PT040371
7989,MAN1B1,MAN1C1,BMG_PT097592;BMG_PT129320;BMG_PT129636;BMG_PT1...,BMG_PT091958;BMG_PT101212;BMG_PT103277
...,...,...,...,...
145760,ACVR2B,NCF2,BMG_PT044531,BMG_PT039496;BMG_PT169028
145762,ACVR2B,RAC1,BMG_PT044531,BMG_PT042017;BMG_PT170932;BMG_PT170933
145764,ACVR2B,RAC2,BMG_PT044531,BMG_PT039170;BMG_PT105276;BMG_PT105347;BMG_PT1...
145766,ACVR2B,RAC3,BMG_PT044531,BMG_PT041824;BMG_PT139271;BMG_PT139992


In [21]:
kegg_filter = kegg_filter[['From_ID', 'To_ID']]
kegg_filter.dropna(subset=['From_ID'], inplace=True)
kegg_filter.dropna(subset=['To_ID'], inplace=True)
kegg_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51971 entries, 1508 to 145768
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   From_ID  51971 non-null  object
 1   To_ID    51971 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2286685467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kegg_filter.dropna(subset=['From_ID'], inplace=True)
C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\2286685467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kegg_filter.dropna(subset=['To_ID'], inplace=True)


In [22]:
kegg_filter['From_ID'] = kegg_filter['From_ID'].str.split(';')
kegg_filter['To_ID'] = kegg_filter['To_ID'].str.split(';')

kegg_filter = kegg_filter.explode('From_ID')
kegg_filter = kegg_filter.explode('To_ID')

kegg_filter.drop_duplicates(inplace=True)
kegg_filter.reset_index(drop=True, inplace=True)
kegg_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\3724326563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kegg_filter['From_ID'] = kegg_filter['From_ID'].str.split(';')
C:\Users\shunn\AppData\Local\Temp\ipykernel_9296\3724326563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kegg_filter['To_ID'] = kegg_filter['To_ID'].str.split(';')


,From_ID,To_ID
0,BMG_PT078734,BMG_PT072935
1,BMG_PT078734,BMG_PT119125
2,BMG_PT078734,BMG_PT120080
3,BMG_PT078734,BMG_PT120852
4,BMG_PT078734,BMG_PT121322
...,...,...
1967509,BMG_PT044531,BMG_PT139992
1967510,BMG_PT044531,BMG_PT039120
1967511,BMG_PT044531,BMG_PT107763
1967512,BMG_PT044531,BMG_PT111094


### Protein-Protein Relation

In [23]:
biomedgraphica_biogrid_ppi = df_biogrid_filter_relation_final.copy()
biomedgraphica_biogrid_ppi['Source_1'] = 'BioGrid'

biomedgraphica_stringdb_ppi = df_stringdb_filter.copy()
biomedgraphica_stringdb_ppi['Source_2'] = 'STRING'

biomedgraphica_kegg_ppi = kegg_filter.copy()
biomedgraphica_kegg_ppi['Source_3'] = 'KEGG'

biomedgraphica_ppi = pd.merge(biomedgraphica_biogrid_ppi, biomedgraphica_stringdb_ppi, how='outer')
biomedgraphica_ppi = pd.merge(biomedgraphica_ppi, biomedgraphica_kegg_ppi, how='outer')
biomedgraphica_ppi

,From_ID,To_ID,Source_1,Source_2,Source_3
0,BMG_PT000170,BMG_PT033607,NaN,STRING,NaN
1,BMG_PT000170,BMG_PT035920,NaN,STRING,NaN
2,BMG_PT000170,BMG_PT036346,NaN,STRING,NaN
3,BMG_PT000170,BMG_PT038694,NaN,STRING,NaN
4,BMG_PT000170,BMG_PT040925,NaN,STRING,NaN
...,...,...,...,...,...
16484815,BMG_PT173978,BMG_PT169819,NaN,NaN,KEGG
16484816,BMG_PT173978,BMG_PT172701,NaN,NaN,KEGG
16484817,BMG_PT173978,BMG_PT173384,NaN,NaN,KEGG
16484818,BMG_PT173978,BMG_PT173385,NaN,NaN,KEGG


In [24]:
def merge_string_columns(df, columns, merge_name, separator=';'):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                combined.update(row[column].split(separator))
        return separator.join(combined)
    
    # Apply the function to each row and create a new column
    combined_column_name = merge_name
    df[combined_column_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)
    
    return df

biomedgraphica_ppi = merge_string_columns(biomedgraphica_ppi, ['Source_1', 'Source_2', 'Source_3'], 'Source')
biomedgraphica_ppi

,From_ID,To_ID,Source
0,BMG_PT000170,BMG_PT033607,STRING
1,BMG_PT000170,BMG_PT035920,STRING
2,BMG_PT000170,BMG_PT036346,STRING
3,BMG_PT000170,BMG_PT038694,STRING
4,BMG_PT000170,BMG_PT040925,STRING
...,...,...,...
16484815,BMG_PT173978,BMG_PT169819,KEGG
16484816,BMG_PT173978,BMG_PT172701,KEGG
16484817,BMG_PT173978,BMG_PT173384,KEGG
16484818,BMG_PT173978,BMG_PT173385,KEGG


In [25]:
biomedgraphica_ppi['Type'] = 'Protein-Protein'
max_length = len(str(len(biomedgraphica_ppi)))
biomedgraphica_ppi['BioMedGraphica_ID'] = ['BMG_ED_PTPT' + str(i).zfill(max_length) for i in range(1, len(biomedgraphica_ppi) + 1)]
columns = ['BioMedGraphica_ID'] + [col for col in biomedgraphica_ppi.columns if col != 'BioMedGraphica_ID']  # re-order columns
biomedgraphica_ppi = biomedgraphica_ppi[columns]
biomedgraphica_ppi

,BioMedGraphica_ID,From_ID,To_ID,Source,Type
0,BMG_ED_PTPT00000001,BMG_PT000170,BMG_PT033607,STRING,Protein-Protein
1,BMG_ED_PTPT00000002,BMG_PT000170,BMG_PT035920,STRING,Protein-Protein
2,BMG_ED_PTPT00000003,BMG_PT000170,BMG_PT036346,STRING,Protein-Protein
3,BMG_ED_PTPT00000004,BMG_PT000170,BMG_PT038694,STRING,Protein-Protein
4,BMG_ED_PTPT00000005,BMG_PT000170,BMG_PT040925,STRING,Protein-Protein
...,...,...,...,...,...
16484815,BMG_ED_PTPT16484816,BMG_PT173978,BMG_PT169819,KEGG,Protein-Protein
16484816,BMG_ED_PTPT16484817,BMG_PT173978,BMG_PT172701,KEGG,Protein-Protein
16484817,BMG_ED_PTPT16484818,BMG_PT173978,BMG_PT173384,KEGG,Protein-Protein
16484818,BMG_ED_PTPT16484819,BMG_PT173978,BMG_PT173385,KEGG,Protein-Protein


In [26]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Relation' / 'Protein-Protein'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Protein_Protein.csv'
biomedgraphica_ppi.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Relation\Protein-Protein\BioMedGraphica_Protein_Protein.csv
